# Import and Setup

In [1]:
# Clone the entire repo
%rm -r graphegfr/
%rm -r configs/
%rm -rf .proj.temp/
!git clone -l -s https://github.com/manbaritone/GraphEGFR .proj.temp/
%rm .proj.temp/run.ipynb
%mv .proj.temp/* ./
%rm -rf .proj.temp/

rm: cannot remove 'graphegfr/': No such file or directory
rm: cannot remove 'configs/': No such file or directory
Cloning into '.proj.temp'...
remote: Enumerating objects: 175, done.
remote: Counting objects: 100% (175/175), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 175 (delta 100), reused 123 (delta 60), pack-reused 0
Receiving objects: 100% (175/175), 523.59 KiB | 9.52 MiB/s, done.
Resolving deltas: 100% (100/100), done.


In [1]:
!pip install -q condacolab
import condacolab
import sys
condacolab.install()
!echo $PYTHONPATH
%env PYTHONPATH=/usr/local/bin/python3.10

✨🍰✨ Everything looks OK!
/env/python
env: PYTHONPATH=/usr/local/bin/python3.10
✨🍰✨ Everything looks OK!


In [ ]:
import condacolab
condacolab.check()

In [2]:
# Install Python dependencies
!pip install scikit-learn==1.2.2
!pip install imblearn
!conda install conda-forge::openbabel
!conda install pytorch==2.0.0 torchvision==0.15.0 torchaudio==2.0.0 -c pytorch
!pip install dgl -f https://data.dgl.ai/wheels/repo.html
!pip install dgllife
!python -m pip install rdkit
!python -m pip install deepchem==2.5.0
!python -m pip install JPype1
!python -m pip install torch_geometric
!python -m pip install torchmetrics

Channels:
 - anaconda
 - conda-forge
Platform: linux-64
Solving environment: | / - done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.4.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - scikit-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blas-1.0                   |         openblas          48 KB  anaconda
    certifi-2023.11.17         |  py310h06a4308_0         159 KB  anaconda
    joblib-1.2.0               |  py310h06a4308_0         398 KB  anaconda
    libgfortran-ng-11.2.0      |       h00389a5_1          19 KB  anaconda
    libgfortran5-11.2.0        |       h1234567_1         5.0 MB  anaconda
    libopenblas-0.3.21         |       h043d6bf_0         9.7 MB  anaconda
    numpy-1.26.0  

In [5]:
# Download large required project files for the test
!wget https://zenodo.org/records/11118070/files/GraphEGFR.tar.gz
!tar -xvf /content/GraphEGFR.tar.gz --warning=no-unknown-keyword -C .
!mv GraphEGFR/* ./

--2024-05-06 11:54:25--  https://zenodo.org/records/11118070/files/GraphEGFR.tar.gz
Resolving zenodo.org (zenodo.org)... 188.184.98.238, 188.184.103.159, 188.185.79.172, ...
Connecting to zenodo.org (zenodo.org)|188.184.98.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7538735522 (7.0G) [application/octet-stream]
Saving to: ‘GraphEGFR.tar.gz’

GraphEGFR.tar.gz    100%[===================>]   7.02G  19.0MB/s    in 6m 28s  

2024-05-06 12:00:54 (18.5 MB/s) - ‘GraphEGFR.tar.gz’ saved [7538735522/7538735522]

._GraphEGFR
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.macl'
GraphEGFR/
GraphEGFR/misc/
GraphEGFR/._.DS_Store
tar: Ignoring unknown extended header keyword 'SCHILY.fflags'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.FinderInfo'
GraphEGFR/.DS_Store
GraphEGFR/state_dict/
GraphEGFR/resources/
GraphEGFR/models/
GraphEGFR/mtl/
GraphEGFR/mtl/._MTL-ALL-MT_MTTL.pt
tar: Ignoring unknown extended header 

In [ ]:
!echo $PYTHONPATH

In [9]:
import pandas as pd
import os
import torch
from torch_geometric.loader import DataLoader

from graphegfr.models import GraphEGFR
from graphegfr.configs import Configs
from graphegfr.fingerprint import Fingerprint
from graphegfr.featurizer import generate_npdata, clean_smiles
from graphegfr.dataset import load_dataset

In [10]:
target_dict = {
    r'MTL_HER124': ['HER1','HER2','HER4'],
    r'MTL_ALL_WT_MT': ['HER1','HER2','HER4','T790M_L858R','L858R','delE746_A750','T790M'],
    r'MTL_HER1_ALL_MT': ['HER1','T790M_L858R','L858R','delE746_A750','T790M'],
    r'MTL_ALL_MT': ['T790M_L858R','L858R','delE746_A750','T790M'],
    r'HER1':['HER1'],
    r'HER2':['HER2'],
    r'HER4':['HER4'],
    r'T790M_L858R':['T790M_L858R'],
    r'L858R':['L858R'],
    r'delE746_A750':['delE746_A750'],
    r'T790M':['T790M']
}

# Main Section

In [15]:
# enter target here
target = "T790M_L858R"
print_architecture = False

datapath = ".temp" # path to save generated fingerprint
smiles_path = f"resources/LigEGFR/data/{target}.csv"

smiles_raw = pd.read_csv(smiles_path)["SMILES_NS"] # series of SMILES
smiles = clean_smiles(smiles_raw).iloc[:5]
for smi in smiles.tolist():
    print(smi)

Number of defect: 0
C#Cc1cccc(Nc2ncnc3cc(OCCOC)c(OCCOC)cc23)c1
C#Cc1nn([C@@H]2CCCN(C(=O)C=C)C2)c2ncnc(N)c12
C=C=CC(=O)Nc1cc(Nc2ncc(C)c(-c3cn(C)c4ccccc34)n2)c(OC)cc1N(C)CCN(C)C
C=C=CC(=O)Nc1cc(Nc2ncc(OC)c(-c3cn(C)c4ccccc34)n2)c(OC)cc1N(C)CCN(C)C
C=C=CC(=O)Nc1cc(Nc2nccc(-c3c[nH]c4ccccc34)n2)c(OC)cc1N(C)CCN(C)C


In [16]:
print("Generating fingerprint...")
Fingerprint(smiles, datapath)
adj, feature, graph, edge = generate_npdata(smiles, datapath)

print("Done")
# print(adj.shape)
# print(feature.shape)
# print(graph.shape)
# print(len(edge))

Generating fingerprint...


0it [00:00, ?it/s]
100%|██████████| 5/5 [00:00<00:00, 21.97it/s]
0it [00:00, ?it/s]
100%|██████████| 5/5 [00:00<00:00, 1872.46it/s]


Done


In [17]:
# Generated from the above cell
fpc = pd.read_csv(f'{datapath}/fingerprint-nonhash.csv').to_numpy()
fpf = pd.read_csv(f'{datapath}/fingerprint-hash.csv').to_numpy()

fingfeaf=fpf.shape[-1]
fingfeac=fpc.shape[-1]
fpfs=[]
fpcs=[]
Label = [None] * len(edge)
for i in fpf:
    fpfs.append(torch.FloatTensor(i))
for i in fpc:
    fpcs.append(torch.FloatTensor(i).unsqueeze(1))

In [18]:
dataset, smiles_list = load_dataset(adj, smiles, Label, fpfs, fpcs)

In [19]:
test_loader = DataLoader(dataset,batch_size=1,shuffle=False)

In [20]:
# list all possible targets
for i in target_dict:
    print(i)

MTL_HER124
MTL_ALL_WT_MT
MTL_HER1_ALL_MT
MTL_ALL_MT
HER1
HER2
HER4
T790M_L858R
L858R
delE746_A750
T790M


In [21]:
configs = Configs.parse(f"configs/sample/{target}-conf.json")
hpconfig = configs['hyperparam']
num_atom_features = hpconfig["num_atom_features"]
edge_dim = hpconfig["edge_dim"]
fingerprint_dim = hpconfig["fingerprint_dim"]
num_layers = hpconfig["num_layers"]
num_timesteps = hpconfig["num_timesteps"]
dropout = 0 # not used in eval mode regardless

model = GraphEGFR(num_atom_features,edge_dim, fingerprint_dim,
                  num_layers, num_timesteps, dropout, fingfeaf,
                  fingfeac, configs)
state_dict = torch.load(f"./state_dict/{target}.pt")
model.load_state_dict(state_dict)
model.cpu().eval()
if print_architecture:
    print("== Model Architecture ==")
    print("Target:", target)
    print("Model:\n",model)
else:
    print()

In [22]:
records = {"smiles":[]}
if "MTL" not in target:
    index_ans = target_dict[configs["target"]].index(target)
    actual_target = target_dict[configs["target"]][index_ans]
    records[actual_target] = []
else:
    for t in target_dict[configs["target"]]:
        records[t] = []
for data, smi in zip(test_loader, smiles_list):
    if "MTL" not in target:
        value = model(data)[0,index_ans].item()
        records[actual_target].append(value)
    else:
        value = model(data)[0,:].tolist()
        for i, v in enumerate(value):
            records[target_dict[configs["target"]][i]].append(v)
    records["smiles"].append(smi)
df_records = pd.DataFrame(records)
df_records

,smiles,T790M_L858R
0,C#Cc1cccc(Nc2ncnc3cc(OCCOC)c(OCCOC)cc23)c1,5.054157
1,C#Cc1nn([C@@H]2CCCN(C(=O)C=C)C2)c2ncnc(N)c12,6.057258
2,C=C=CC(=O)Nc1cc(Nc2ncc(C)c(-c3cn(C)c4ccccc34)n...,8.891642
3,C=C=CC(=O)Nc1cc(Nc2ncc(OC)c(-c3cn(C)c4ccccc34)...,8.990313
4,C=C=CC(=O)Nc1cc(Nc2nccc(-c3c[nH]c4ccccc34)n2)c...,8.798269
